In [71]:
import vowpalwabbit
import random
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import environment2

In [72]:
env = environment2.environment2()
user_list_2D = env.user_list_2D
env.process_data(user_list_2D[0], 0.80)

size=len(env.mem_reward)

37 29



In [73]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv'

In [74]:
import os
from os.path import join
import re

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from vowpalwabbit.dftovw import DFtoVW
from vowpalwabbit import Workspace

In [75]:
df = pd.read_csv( 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv')
df=df[df.State !='Answering']

In [76]:
df.head(5)
for i in range(len(df)):
    if df.loc[i, "State"] == 'Navigation':
        df.loc[i, "State"]=0
    elif df.loc[i, "State"] == 'Foraging':
        df.loc[i, "State"]=1
    else:
        df.loc[i, "State"]=2



In [77]:
df.head(5)

,UserIndex,Action,State,TimeStamp,HighLevelStateActionProbab,StateActionProbab,IdleTime,Reward,Subtask_Screenshot_Time,Subtask_Idle_Time,...,Subtask_Zoom,Subtask_ROI,Latitude,Longitude,All_address,All_raw_address,Most_frequent_state,Most_frequent_region,Angle,high-level-action
0,0,start,1,2016-06-10 17:51:46.368,0.666667,0.400000,0.000,0,1,1,...,1,1,34.103050,-98.612800,"['None', 'Other', 'None', 'Other', 'None', 'Ot...","[[15.2153, -132.053], [52.9908, -65.1726], [15...",None,None,0.000000,change
1,1,zoom,1,2016-06-10 17:52:15.238,0.500000,0.333333,28.870,0,1,1,...,1,1,32.654175,-94.199313,"['None', 'South', 'Other', 'None', 'None', 'No...","[[18.1367, -107.299], [30.3858, -97.2569], [15...",None,None,0.000000,same
2,2,pan,1,2016-06-10 17:52:16.021,0.600000,0.285714,0.783,0,1,2,...,1,1,41.766800,-79.535500,"['South', 'Other', 'South', 'Other', 'South', ...","[[30.5342, -93.9559], [52.9994, -65.1151], [30...",Texas,South,352.326038,same
3,3,pan,1,2016-06-10 17:52:18.029,0.666667,0.375000,2.008,0,1,2,...,1,1,41.766800,-79.535500,"['South', 'Other', 'South', 'Other', 'South', ...","[[30.5342, -93.9559], [52.9994, -65.1151], [30...",Texas,South,353.106469,same
4,4,zoom,0,2016-06-10 17:52:18.735,0.666667,0.400000,0.706,0,1,2,...,1,1,47.379675,-110.028137,"['None', 'Other', 'West', 'NorthernRockiesPlai...","[[41.7792, -132.156], [52.9917, -116.51], [41....",Nevada,West,0.000000,change


In [78]:
train_df=df[:round(0.80*len(df))]
test_df=df[round(0.80*len(df)):]
train_df["index"] = range(1, len(train_df) + 1)
train_df = train_df.set_index("index")
test_df["index"] = range(1, len(test_df) + 1)
test_df = test_df.set_index("index")

C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_33584\2238002601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["index"] = range(1, len(train_df) + 1)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_33584\2238002601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["index"] = range(1, len(test_df) + 1)


In [79]:
test_df.head(5)

,UserIndex,Action,State,TimeStamp,HighLevelStateActionProbab,StateActionProbab,IdleTime,Reward,Subtask_Screenshot_Time,Subtask_Idle_Time,...,Subtask_Zoom,Subtask_ROI,Latitude,Longitude,All_address,All_raw_address,Most_frequent_state,Most_frequent_region,Angle,high-level-action
index,,,,,,,,,,,,,,,,,,,,,
1,70,pan,1,2016-06-10 17:54:13.279,0.650000,0.772727,0.573,0,2,2,...,2,6,50.748288,-108.633025,"['None', 'Other', 'NorthernRockiesPlains', 'No...","[[48.5013, -125.495], [52.9961, -112.008], [48...",Montana,NorthernRockiesPlains,3.011003,same
2,71,pan,1,2016-06-10 17:54:14.130,0.666667,0.782609,0.851,0,2,2,...,2,6,38.312137,-96.623625,"['None', 'None', 'NorthWest', 'None', 'NorthWe...","[[49.1719, -132.156], [52.9938, -125.507], [49...",None,None,349.374868,same
3,72,pan,1,2016-06-10 17:54:16.233,0.681818,0.791667,2.103,0,2,2,...,2,6,51.082850,-128.831500,"['None', 'NorthWest', 'None', 'NorthWest', 'No...","[[49.1719, -132.156], [52.9938, -125.507], [49...",None,None,5.736687,same
4,73,pan,1,2016-06-10 17:54:16.747,0.695652,0.800000,0.514,0,2,2,...,2,6,49.341163,-105.256025,"['SouthWest', 'NorthernRockiesPlains', 'Northe...","[[41.7504, -111.985], [48.49, -98.502], [48.5,...",Montana,NorthernRockiesPlains,298.885459,same
5,74,pan,1,2016-06-10 17:54:18.946,0.708333,0.807692,2.199,0,2,2,...,2,6,49.153650,-93.617475,"['NorthernRockiesPlains', 'NorthernRockiesPlai...","[[41.7644, -98.4982], [48.4885, -88.6355], [48...",Ontario,NorthernRockiesPlains,294.129458,same


In [94]:
vw = vowpalwabbit.Workspace("--cb_explore 3 --bag 5 ")

# use the learn method to train the vw model, train model row by row using a loop
for i in train_df.index:
    ## provide data to cb in requested format
    action = train_df.loc[i, "State"]
    cost = -1 * train_df.loc[i, "NDSI"]
    probability = train_df.loc[i, "HighLevelStateActionProbab"]
    feature1 = train_df.loc[i, "Most_frequent_region"]

    ## do the actual learning
    vw.learn(
        str(action)
        + ":"
        + str(cost)
        + ":"
        + str(probability)
        + " | "
        + str(feature1)
    )


In [95]:
for j in test_df.index:
    feature1 = test_df.loc[j, "Most_frequent_region"]
    choice = vw.predict(
        "| " + str(feature1)
    )
    print(j, choice)

1 [0.20000000298023224, 0.800000011920929, 0.0]
2 [1.0, 0.0, 0.0]
3 [1.0, 0.0, 0.0]
4 [0.20000000298023224, 0.800000011920929, 0.0]
5 [0.20000000298023224, 0.800000011920929, 0.0]
6 [0.20000000298023224, 0.800000011920929, 0.0]
7 [0.20000000298023224, 0.800000011920929, 0.0]
8 [0.20000000298023224, 0.800000011920929, 0.0]
9 [0.20000000298023224, 0.800000011920929, 0.0]
10 [0.20000000298023224, 0.800000011920929, 0.0]
11 [1.0, 0.0, 0.0]
12 [0.20000000298023224, 0.800000011920929, 0.0]
13 [0.20000000298023224, 0.800000011920929, 0.0]
14 [0.20000000298023224, 0.800000011920929, 0.0]
15 [0.20000000298023224, 0.800000011920929, 0.0]
16 [0.4000000059604645, 0.6000000238418579, 0.0]
17 [0.4000000059604645, 0.6000000238418579, 0.0]


In [4]:
def get_cost(index):
    return env.mem_reward[index]

In [5]:
# This function modifies (context, action, cost, probability) to VW friendly format
def to_vw_example_format(context, actions, cb_label=None):
    if cb_label is not None:
        chosen_action, cost, prob = cb_label
    example_string = ""
    example_string += "shared |State state={} time={}\n".format(
        context["state"], context["time"]
    )
    for action in actions:
        if cb_label is not None and action == chosen_action:
            example_string += "0:{}:{} ".format(cost, prob)
        example_string += "|Action switch/stay={} \n".format(action)
    # Strip the last newline
    return example_string[:-1]

In [6]:
context = {"state": "Sensemaking", "time": 1}
actions = ["same", "change"]

print(to_vw_example_format(context, actions))

shared |State state=Sensemaking time=1
|Action switch/stay=same 
|Action switch/stay=change 


In [7]:
def sample_custom_pmf(pmf):
    total = sum(pmf)
    scale = 1 / total
    pmf = [x * scale for x in pmf]
    draw = random.random()
    sum_prob = 0.0
    for index, prob in enumerate(pmf):
        sum_prob += prob
        if sum_prob > draw:
            return index, prob

In [8]:
def get_action(vw, context, actions):
    vw_text_example = to_vw_example_format(context, actions)
    pmf = vw.predict(vw_text_example)
    chosen_action_index, prob = sample_custom_pmf(pmf)
    return actions[chosen_action_index], prob

In [9]:
states = ["Foraging","Navigation", "Sensemaking"]
time = ["morning", "afternoon"]
actions = ["same","change"]


def choose_user(states):
    return random.choice(states)


def choose_time_of_day(time):
    return random.choice(time)


# display preference matrix
def get_preference_matrix(cost_fun):
    def expand_grid(data_dict):
        rows = itertools.product(*data_dict.values())
        return pd.DataFrame.from_records(rows, columns=data_dict.keys())

    df = expand_grid({"states": states, "time": time, "actions": actions})
    df["cost"] = df.apply(
        lambda r: cost_fun(1), axis=1
    )

    return df.pivot_table(
        index=["states", "time"], columns="actions", values="cost"
    )


get_preference_matrix(get_cost)

actions                  change      same
states      time                         
Foraging    afternoon  0.626178  0.626178
            morning    0.626178  0.626178
Navigation  afternoon  0.626178  0.626178
            morning    0.626178  0.626178
Sensemaking afternoon  0.626178  0.626178
            morning    0.626178  0.626178

In [71]:
def run_simulation(
    vw, num_iterations, states, times, actions, cost_function, do_learn=True
):
    cost_sum = 0.0
    ctr = []
    cnt=0

    for i in range(1, num_iterations + 1):

            # 1. In each simulation choose a user
            state = choose_user(states)
            # 2. Choose time of day for a given user
            time = choose_time_of_day(times)

            # 3. Pass context to vw to get an action
            context = {"state": state, "time": time}
            action, prob = get_action(vw, context, actions)

            # 4. Get cost of the action we chose
            cost = cost_function(cnt)
            cost_sum += cost
            cnt+=1

            if do_learn:
                # 5. Inform VW of what happened so we can learn from it
                vw_format = vw.parse(
                    to_vw_example_format(context, actions, (action, cost, prob)),
                    vowpalwabbit.LabelType.CONTEXTUAL_BANDIT,
                )
                # 6. Learn
                vw.learn(vw_format)

            # We negate this so that on the plot instead of minimizing cost, we are maximizing reward
            ctr.append(-1 * cost_sum / i)

    return ctr

In [72]:
def plot_ctr(num_iterations, ctr):
    plt.plot(range(1, num_iterations + 1), ctr)
    plt.xlabel("num_iterations", fontsize=14)
    plt.ylabel("ctr", fontsize=14)
    plt.ylim([0, 1])

In [73]:
# Instantiate learner in VW
vw = vowpalwabbit.Workspace("--cb_explore_adf -q UA --quiet --epsilon 0.2")

num_iterations = 5000
ctr = run_simulation(vw, num_iterations, states, time, actions, get_cost)

plot_ctr(num_iterations, ctr)

IndexError: list index out of range